In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import os
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import json
import random
import shutil
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.inception_v3 import InceptionV3

In [ ]:
source_dir = '../input/cassava-leaf-disease-classification/train_images'
all_images = os.listdir(source_dir)
data_label =  pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
label_name_path = '../input/cassava-leaf-disease-classification/label_num_to_disease_map.json'
with open(label_name_path) as f:
    label_name = json.load(f)

In [ ]:
def load_data(source_dir, img_height, img_width, train_size=0.9):
    X_train = []
    y_train = []
    X_test = []
    y_test = []
    random.shuffle(all_images)
    split_size = int(len(all_images)*train_size)
    #split_size = 100
    train_images = all_images[0:split_size]
    test_images = all_images[split_size:]
    for img in train_images:
        x = cv2.resize(cv2.imread(os.path.join(source_dir, img)), (img_height, img_width))
        X_train.append(x)
        y_train.append(data_label[data_label['image_id'] == img]['label'].values[0])
    for img in test_images:
        x = cv2.resize(cv2.imread(os.path.join(source_dir, img)), (img_height, img_width))
        X_test.append(x)
        y_test.append(data_label[data_label['image_id'] == img]['label'].values[0])
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    X_test = np.array(X_test)
    y_test = np.array(y_test)
    return X_train, y_train, X_test, y_test

In [ ]:
X_train, y_train, X_test, y_test = load_data(source_dir, 100, 100, 1)
y_train = to_categorical(y_train, 5)
y_test = to_categorical(y_test, 5)

In [ ]:
datagen = ImageDataGenerator(rescale=1/255.)
train_generator = datagen.flow(X_train,y_train,
                                batch_size=64)
#datagen_test = ImageDataGenerator(rescale=1/255.)
#test_generator = datagen_test.flow(X_test,y_test, batch_size=64)

In [ ]:
def define_directory():
    os.mkdir('/kaggle/working/Training')
    os.mkdir('/kaggle/working/Training/CBB')
    os.mkdir('/kaggle/working/Training/CBSD')
    os.mkdir('/kaggle/working/Training/CGM')
    os.mkdir('/kaggle/working/Training/CMD')
    os.mkdir('/kaggle/working/Training/Healthy')

In [ ]:
#define_directory()
#target_path_dict = {0:'CBB', 1:'CBSD', 2:'CGM', 3:'CMD', 4:'Healthy'}
#training_dir = '/kaggle/working/Training'

In [ ]:
#for label in label_name:
#    images = data_label[data_label['label'] == int(label)]['image_id'].values
#    for img in images:
#        source = os.path.join(source_dir, img)
#        target = os.path.join(training_dir, target_path_dict[int(label)])
#        shutil.copy(source, target)

In [ ]:
#img_height = 150
#img_width = 150
#batch_size = 32
#train_data_gen = ImageDataGenerator(rescale=1/255.)
#train_generator = train_data_gen.flow_from_directory(training_dir,
#                                                     target_size = (img_height, img_width),
#                                                   batch_size=batch_size)

In [ ]:
#pre_trained_model = VGG16(include_top=False, weights=None, input_shape=(100, 100, 3))
#pre_trained_model.load_weights('../input/vgg-16-weight-file/vgg16_weight_file.h5')
pre_trained_model = InceptionV3(include_top=False, weights=None, input_shape=(100, 100, 3))
pre_trained_model.load_weights('../input/inceptionv3-weights/inception_v3_weights.h5')
for layer in pre_trained_model.layers:
    layer.trainable = False
last_layer = pre_trained_model.get_layer('mixed7')
last_output = last_layer.output
#Define model
x = tf.keras.layers.Flatten()(last_output)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
#x = tf.keras.layers.Dense(2048, activation='relu')(x)
#x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(5, activation='softmax')(x)
model = tf.keras.Model(pre_trained_model.input, x)
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.004), loss='categorical_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = model.fit(train_generator, epochs=30)

In [ ]:
#plt.plot(history.history['accuracy'])
#plt.plot(history.history['val_accuracy'])
#plt.show()

# Prediction

In [ ]:
test_dir = '../input/cassava-leaf-disease-classification/test_images'
test_images = os.listdir(test_dir)

In [ ]:
img_id = []
lbl = []
for img in test_images:
    x = cv2.resize(cv2.imread(os.path.join(test_dir, img)), (100, 100))
    x = np.array(x)
    x = x/255.
    x = np.expand_dims(x, axis=0)
    y = np.argmax(model.predict(x))
    lbl.append(y)
    img_id.append(img)
submission_df = pd.DataFrame(columns=['image_id', 'label'])
submission_df['image_id'] = img_id
submission_df['label'] = lbl
submission_df.to_csv('./submission.csv', index=False)